In [1]:
import os, cv2
import splitfolders
import shutil

In [7]:
from ultralytics import YOLO, SAM
import cv2
import numpy as np
import os

# ===== CONFIG =====
model = YOLO("runs/segment/Newagsolaire_yolo_segmentation/weights/best.pt")
images_folder = "/home/sowmya/AgSolaire/train_val_test_split/test/images"
sam_model = SAM("sam2_b.pt")

PX_PER_MM = None   # e.g. 12.5 if you know calibration; else stays in px

# ===== MAIN LOOP =====
for file in os.listdir(images_folder):
    full_path = os.path.join(images_folder, file)
    if not os.path.isfile(full_path):
        continue

    image = cv2.imread(full_path)
    image = cv2.resize(image, (680, 900))

    # Run YOLO detection
    results = model.predict(source=image, conf=0.8)
    boxes = results[0].boxes.xyxy.cpu().numpy()

    # Run SAM segmentation using YOLO boxes as prompts
    sam_results = sam_model(image, bboxes=boxes)

    overlay = image.copy()
    seed_id = 0

    for r in sam_results:
        masks = r.masks.data.cpu().numpy()  # [N, H, W]

        for mask in masks:
            seed_id += 1
            mask = (mask > 0.5).astype(np.uint8)  # ensure binary mask

            # Find contour
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not contours:
                continue

            cnt = max(contours, key=cv2.contourArea)

            # Fit rotated rectangle
            rect = cv2.minAreaRect(cnt)     # (center, (w,h), angle)
            (cx, cy), (w, h), angle = rect
            length_px = max(w, h)
            width_px  = min(w, h)

            # Convert to mm if calibration is known
            length_mm = length_px / PX_PER_MM if PX_PER_MM else None
            width_mm  = width_px  / PX_PER_MM if PX_PER_MM else None

            # Draw rectangle
            box = cv2.boxPoints(rect)
            box = np.int32(box)
            cv2.polylines(overlay, [box], True, (255, 0, 0), 2)

            # Draw center point
            cv2.circle(overlay, (int(cx), int(cy)), 4, (0, 255, 0), -1)

            # Put text label (both length and width)
            if length_mm and width_mm:
                label = f"{seed_id}: {length_mm:.2f} x {width_mm:.2f} mm"
            else:
                label = f"{seed_id}: {length_px:.1f} x {width_px:.1f} px"

            cv2.putText(overlay, label, (int(cx)+10, int(cy)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    # Blend overlay with original image
    out = cv2.addWeighted(image, 0.7, overlay, 0.3, 0)

    cv2.imshow("Seeds with Lengths & Widths", out)
    cv2.waitKey(0)

cv2.destroyAllWindows()



0: 480x384 26 seed masks, 10.9ms
Speed: 1.0ms preprocess, 10.9ms inference, 6.4ms postprocess per image at shape (1, 3, 480, 384)

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1 18, 1 19, 1 20, 1 21, 1 22, 1 23, 1 24, 1 25, 703.3ms
Speed: 3.6ms preprocess, 703.3ms inference, 2.3ms postprocess per image at shape (1, 3, 1024, 1024)

0: 480x384 26 seed masks, 10.1ms
Speed: 1.1ms preprocess, 10.1ms inference, 6.1ms postprocess per image at shape (1, 3, 480, 384)

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1 18, 1 19, 1 20, 1 21, 1 22, 1 23, 1 24, 1 25, 722.7ms
Speed: 3.5ms preprocess, 722.7ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 480x384 26 seed masks, 9.5ms
Speed: 1.0ms preprocess, 9.5ms inference, 5.7ms postprocess per image at shape (1, 3, 480, 384)

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 1

In [8]:
from ultralytics import YOLO, SAM
import cv2
import numpy as np
import os

# ===== CONFIG =====
model = YOLO("/home/sowmya/Downloads/best.pt")
images_folder = "/home/sowmya/AgSolaire/train_val_test_split/test/images"
sam_model = SAM("sam2_b.pt")

PX_PER_MM = None   # e.g. 12.5 if you know calibration; else stays in px

# ===== MAIN LOOP =====
for file in os.listdir(images_folder):
    full_path = os.path.join(images_folder, file)
    if not os.path.isfile(full_path):
        continue

    image = cv2.imread(full_path)
    image = cv2.resize(image, (1400, 1120))

    # Run YOLO detection
    results = model.predict(source=image, conf=0.7)
    boxes = results[0].boxes.xyxy.cpu().numpy()

    # Run SAM segmentation using YOLO boxes as prompts
    sam_results = sam_model(image, bboxes=boxes)

    overlay = image.copy()
    seed_id = 0

    for r in sam_results:
        masks = r.masks.data.cpu().numpy()  # [N, H, W]

        for mask in masks:
            seed_id += 1
            mask = (mask > 0.5).astype(np.uint8)  # ensure binary mask

            # Find contour
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not contours:
                continue

            cnt = max(contours, key=cv2.contourArea)

            # ---- Fit ellipse instead of rectangle ----
            if len(cnt) < 5:   # cv2.fitEllipse requires >=5 points
                continue

            ellipse = cv2.fitEllipse(cnt)   # (center(x,y), (major_axis, minor_axis), angle)
            (cx, cy), (MA, ma), angle = ellipse

            # Major = length, Minor = width
            length_px = max(MA, ma)
            width_px  = min(MA, ma)

            # Convert to mm if calibration known
            length_mm = length_px / PX_PER_MM if PX_PER_MM else None
            width_mm  = width_px  / PX_PER_MM if PX_PER_MM else None

            # ---- Draw ellipse ----
            cv2.ellipse(overlay, ellipse, (0, 255, 0), 2)  # blue ellipse
            cv2.circle(overlay, (int(cx), int(cy)), 4, (0, 255, 0), -1)  # green center

            # Put text label
            if length_mm and width_mm:
                label = f"{seed_id}: {length_mm:.2f} x {width_mm:.2f} mm"
            else:
                label = f"{seed_id}: {length_px:.1f} x {width_px:.1f} px"

            # cv2.putText(overlay, label, (int(cx)+10, int(cy)-10),
                        # cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    # Blend overlay with original image
    out = cv2.addWeighted(image, 0.7, overlay, 0.3, 0)

    cv2.imshow("Seeds with Lengths & Widths (Ellipses)", out)
    cv2.waitKey(0)

cv2.destroyAllWindows()



0: 512x640 33 seed masks, 46.7ms
Speed: 1.7ms preprocess, 46.7ms inference, 8.1ms postprocess per image at shape (1, 3, 512, 640)

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1 18, 1 19, 1 20, 1 21, 1 22, 1 23, 1 24, 1 25, 1 26, 1 27, 1 28, 1 29, 1 30, 1 31, 1 32, 750.2ms
Speed: 4.6ms preprocess, 750.2ms inference, 5.6ms postprocess per image at shape (1, 3, 1024, 1024)

0: 512x640 31 seed masks, 43.2ms
Speed: 1.7ms preprocess, 43.2ms inference, 7.7ms postprocess per image at shape (1, 3, 512, 640)

0: 1024x1024 1 0, 1 1, 1 2, 1 3, 1 4, 1 5, 1 6, 1 7, 1 8, 1 9, 1 10, 1 11, 1 12, 1 13, 1 14, 1 15, 1 16, 1 17, 1 18, 1 19, 1 20, 1 21, 1 22, 1 23, 1 24, 1 25, 1 26, 1 27, 1 28, 1 29, 1 30, 751.0ms
Speed: 4.4ms preprocess, 751.0ms inference, 5.4ms postprocess per image at shape (1, 3, 1024, 1024)

0: 512x640 24 seed masks, 38.1ms
Speed: 1.9ms preprocess, 38.1ms inference, 6.4ms postprocess per image at shape (1, 3, 512, 640)